In [36]:
#This gives python access to all the TensorFlow's classes, methods and symbols
import tensorflow as tf
import numpy as np

print(tf.__version__)

1.4.0


Notes
===
* The higher level APIs are built on top of TensorFlow Core.
* Central Unit in TensorFlow is a Tensor. Array of any number of dimensions (ranks)
* A TensorFlow graph is a series of TensorFlow operations arranged into a graph of nodes
* A Node takes zero or more tensors as input and outputs another tensor
* To actually evaluate nodes (produce their ouput) we must run the computational graph within a session
* Operations are also nodes
* Tensorboard can display a picture of the computational graph
* A graph can be parameterized to accept external inputs
* Placeholder: accept external inputs (later)
* Variables are not initialized when you call tf.Variable. To initialize all the varibles in a TensorFlow program, you must explicitly call tf.global_variables_initializer()
* A loss function measure how far apart the current model is from the provided data.
* A variable is initialized to the value provided to tf.Variable but can be changed using operations like tf.assign
* Optimizer: Technique to slowly change each variables in order to minimize the loss function
* tf.estimator: High Level TensorFlow library that simplifies the mechanics of machine learning including the following:
    * running training loops
    * running evaluation loops
    * managing data sets


In [6]:
#Nodes
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0)
print(node1)

Tensor("Const_1:0", shape=(), dtype=float32)


In [7]:
sess = tf.Session()
print(sess.run(node1))

3.0


In [9]:
from __future__ import print_function
node3 = tf.add(node1, node2)
print("node3: ", node3)
print(sess.run(node3))

node3:  Tensor("Add_1:0", shape=(), dtype=float32)
7.0


In [10]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # + is shorcut for tf.add()

In [11]:
#Here we feed values into the place holders
print(sess.run(adder_node, {a:3, b:4.5})) 

7.5


In [12]:
add_and_triple = adder_node * 3
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


In [16]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x+b

In [17]:
init = tf.global_variables_initializer()
sess.run(init)

In [18]:
print(sess.run(linear_model, {x: [1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [22]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1,2,3,4], y: [0, -1, -2, -3]}))

23.66


In [23]:
fixW = tf.assign(W, [-1.]) #Why do we need to store it into fixW?
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1,2,3,4], y: [0, -1, -2, -3]}))

0.0


**tf.train API**

In [28]:
optimizer = tf.train.GradientDescentOptimizer(0.01) #Learning rate
train = optimizer.minimize(loss)

sess.run(init) #reset values to incorrect defaults

for i in range(1000):
    sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

print(sess.run([W,b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


**Trainable Linear Regression Model (Low Level)**

In [37]:
import tensorflow as tf

#Model Parameter y = Wx + b
W = tf.Variable(.3, dtype = tf.float32)
b = tf.Variable(.1, dtype = tf.float32)
init = tf.global_variables_initializer()


#Model Input and output y = Wx + b
y = tf.placeholder(tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

#loss
squared_deltas = tf.square(linear_model - y) 
loss = tf.reduce_sum(squared_deltas) #sum of the squares

#optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

#training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

#training loop
with tf.Session() as sess:
    sess.run(init) #Initialized all variables to default
    for i in range(1000): #cycles
        sess.run(train, {x: x_train, y: y_train})
        
    #evaluate training accuracy
    curr_W, curr_b, curr_loss = sess.run([W,b,loss], {x:x_train, y: y_train})
    print("W: %s, b: %s, loss: %s"%(curr_W, curr_b, curr_loss))
    




W: -0.999998, b: 0.999993, loss: 3.18785e-11


The loss is very small number (close to zero)

**Trainable Linear Regression Model (Estimator: High Level)**

In [55]:
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np
import tensorflow as tf

# Declare list of features
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation (inference)
# There are many predefined types like linear regression, linear classification, and
# many neural networks classifiers and regressors.

estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# We have to tell the function how many batches of data (num_epochs)
# we want and how big each batch should be

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1, -2, -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
x_test = np.array([1., 2., 3., 4.])

input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=True)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

test_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_test}, y= None, batch_size=4, num_epochs=1, shuffle=False)


# We can invoke 1000 training steps by invoking the method and passing the
# training dataset

estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
test_predict = estimator.predict(input_fn=test_input_fn)

print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)
print("\n")

sess = tf.Session()

for i in test_predict:
    print(i)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\avour\\AppData\\Local\\Temp\\tmpcfvakvzt', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D4A3148F28>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\avour\AppData\Local\Temp\tmpcfvakvzt\model.ckpt.
INFO:tensorflow:loss = 19.0, step = 1
INFO:tensorflow:global_step/sec: 859.769
INFO:tensorflow:loss = 0.115248, step = 101 (0.117 sec)
INFO:tensorflow:global_step/sec: 1028.19
INFO:tensorflow:loss = 0.0502527, step = 201 (0.097 sec)
I